In [1]:
import numpy as np
import pandas as pd
import sys
# np.set_printoptions(threshold=sys.maxsize)

data = pd.read_csv('train.csv')
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
data = np.array(data)
m,n = data.shape # m test digits(rows), n pixels(columns)
np.random.shuffle(data)

data_dev = data[0:1000].T # transpose data matrix for ease of use, 784xm
Y_dev = data_dev[0] # 1xm matrix with labels, indicates correct answer
X_dev = data_dev[1:n] # 784xm matrix with image data, each column is one digit
X_dev = X_dev / 255

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255

### Remark.

    The dataset is split into two parts in order to train the model on the larger part and test the model on the smaller one. This prevents the model from overfitting to the training data.

In [3]:
def initWB(n_h1,n_h2):
    # W1 = np.random.rand(n_h1, 784) # n_h1x784 matrix with random weights from [0,1) (uniformly chosen)
    # b1 = np.random.rand(n_h1, 1) # n_h1x1 vector with random biases from [0,1)
    # W2 = np.random.rand(n_h2, n_h1) # n_h2xn_h1
    # b2 = np.random.rand(n_h2, 1) # n_h2x1
    W1 = np.random.rand(n_h1, 784) - 0.5
    b1 = np.random.rand(n_h1, 1) - 0.5
    W2 = np.random.rand(n_h2, n_h1) - 0.5
    b2 = np.random.rand(n_h2, 1) - 0.5
    return W1, b1, W2, b2

### Observation.

    Initializing the weights and biases in the range of [0.5,0.5) results in much faster learning than initializing them in the range of [0,1). 

### Thoughts.

    This may have to do with the fact that weights and biases get blown up throught the iterations if they are always positive. Having negative values may help the gradient descent happen more smoothly.

In [4]:
def ReLU(Z):
    return np.maximum(Z, 0)

def ReLU_deriv(Z):
    return (Z>0).astype(float)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return expZ / expZ.sum(axis=0, keepdims=True)

def feedForward(W1,b1,W2,b2,X):
    Z1 = np.matmul(W1,X)+b1 # n_h1xm matrix of weighted sum of previous nodes 
    A1 = ReLU(Z1) # n_h1xm
    Z2 = np.matmul(W2,A1)+b2 # n_h2xm 
    A2 = softmax(Z2) # n_h2xm
    # A2 = ReLU(Z2)
    return Z1,A1,Z2,A2

<center>
    <img src="diagrams/IMG_4953.png" width="500" alt="super simple NN"/>
</center>

(IMG_4953)

### The Math.

The following shows how feeding forward in neural networks works. $z$ denotes the weighted sum of the previous activations with respect to weights plus the bias. $a$ denotes the weighted sum $z$ ran through a non-linear activation function $\sigma$ (this is important as to not have each activation simply be a linear combination of the inputs). $w$ denotes the weights between two activations and $b$ denotes the bias for each layer.

$$z^{(L)}_j = \sum_{n=1}^{m} a^{(L-1)}_n*w^{(L)}_{jn} + b^{(L)}$$
$$a^{(L)}_j = \sigma(z^{(L)}_j)$$

The superscripts denote the layer the node is in (or, for the weights, what layer they are heading to) and the subscripts denote the index number within the layer. For example, $$z^{(L)}_j$$ denotes the weighted sum of the $j$th node in the $L$th layer. Additionally, $$w^{(L)}_{jn}$$ denotes the weight of the edge connecting the $n$th node in the $L-1$th layer to the $j$th node in the $L$th layer. The component-wise form can be condensed into a matrix-form equation which makes the notation and code much simpler.

$$A^{(L)} = \sigma(W^{(L)}A^{(L-1)}+B^{(L)})$$

### Observation.

Using ReLU for the hidden layer and softmax for the output layer results in the fastest learning speed -- the difference is pretty dramatic with using the softmax resulting in accuracy almost 6 times that of using just the ReLU. Without the softmax, the activation function for the final layer effectively becomes the argmax function (since ReLU returns the input for any positive input) which simply returns the largest value in the output vector. 

### Thoughts.

I think the process of multiplying the derivative of ReLU for delta2 in the process of backpropagation may result in a lot of zeros, which slows down gradient descent.

In [5]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, 10))
    for i in range(0,Y.size):
        one_hot_Y[i,Y[i]]=1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y
    
def backProp(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = X.shape[1] # number of test digits
    Y = one_hot(Y)
    
    delta2 = (A2-Y) # n_h2xm matrix
    # delta2 = (A2-Y)*ReLU_deriv(Z2)
    dW2 = np.matmul(delta2,A1.T)/m # n_h2xn_h1 matrix, matrix multiplying inherently involves summing up all test case errors which means division by m is necessary
    db2 = np.sum(delta2,axis=1,keepdims=True)/m # n_h2x1 matrix
    
    delta1 = np.matmul(W2.T,delta2)*ReLU_deriv(Z1) # n_h1xm matrix
    dW1 = np.matmul(delta1,X.T)/m # n_h1xm matrix
    db1 = np.sum(delta1,axis=1,keepdims=True)/m # n_h1x1 matrix
    
    return dW1, dW2, db1, db2

def updateWB(W1, W2, b1, b2, dW1, dW2, db1, db2, alpha):
    W1 -= alpha*dW1
    W2 -= alpha*dW2
    b1 -= alpha*db1
    b2 -= alpha*db2
    return W1,b1,W2,b2
    

### Observation.

The importance of paying attention to the dimensions of the matrices cannot be understated. When turning component-wise math into more compact matrix form notation, the easiest way to keep track of everything is to always keep the dimensions of the matrices in mind. This is especially important when transposes are necessary to make the dimensions match up.

In [6]:
def get_predictions(A2):
    return np.argmax(A2, 0) #returns an array of indices indicating the likliest digit by finding the largest number along each column

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradDesc(X,Y,alpha,iterations):
    W1,b1,W2,b2 = initWB(10,10)
    for i in range(iterations):
        Z1,A1,Z2,A2 = feedForward(W1,b1,W2,b2,X)
        dW1, dW2, db1, db2 = backProp(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1,b1,W2,b2 = updateWB(W1, W2, b1, b2, dW1, dW2, db1, db2, alpha)
        # if(i%10==0):
        #     print("interation: ",i)
        #     predictions = get_predictions(A2)
        #     print(get_accuracy(predictions, Y))
        # if(i==iterations-1):
        #     print("interation: ",i)
        #     predictions = get_predictions(A2)
        #     print(get_accuracy(predictions, Y))
    return W1,b1,W2,b2

def testWB(W1,b1,W2,b2,X,Y):
    Z1,A1,Z2,A2 = feedForward(W1,b1,W2,b2,X)
    predictions = get_predictions(A2)
    print("accuracy for test data:",get_accuracy(predictions, Y))


### Remark.

The process of repeating the feedForward and backpropagation means the neural network "learns" as iterations increase. The weights and biases are nudged according to the backpropagation algorithm such that the cost function decreases. More specifically, the gradient of the cost function is calculated with respect to all the weights and the weights are nudged in proportion to this gradient (the same is done for the biases). Thus, the weights and biases become more and more fine tuned to the pattern recognition that is necessary.

### The Math.

Let the $L$th layer be the last layer, i.e. the output layer. Then,

$$\frac{ \partial{C} } { \partial{w^{(L)}_{jk}} } = 
\frac{ \partial{z^{(L)}_j} } { \partial{w^{(L)}_{jk}} }
\frac{ \partial{a^{(L)}_j} } { \partial{z^{(L)}_j} }
\frac{ \partial{C} } { \partial{a^{(L)}_j} }
$$

$$z^{(L)}_j = \sum_{n=1}^{m} a^{(L-1)}_n*w^{(L)}_{jn} + b_j$$

$$\implies \frac{ \partial{z^{(L)}_j} } { \partial{w^{(L)}_{jk}} } = a^{(L-1)}_k$$

$$a^{(L)}_j = \sigma(z^{(L)}_j)$$

$$\implies \frac{ \partial{a^{(L)}_j} } { \partial{z^{(L)}_j} } = \sigma\prime(z^{(L)}_j)$$

$$C = \frac{1}{2} \sum_{j=1}^{N}(a^{(L)}_j-y_{j})^2$$

$$\implies \frac{ \partial{C} } { \partial{a^{(L)}_j} } = a^{(L)}_j-y_{j}$$

$$\implies \frac{ \partial{C} } { \partial{w^{(L)}_{jk}} } = a^{(L-1)}_k*\sigma\prime(z^{(L)}_j)*(a^{(L)}_j-y_{j})$$

The process of calculating $\frac{ \partial{C} } { \partial{w^{(l)}_{jk}} }$ isn't as simple for an arbitrary layer $l$ that is not the output layer. Thus, to simplify notation and save some brain power, we define a new function denoted by $\delta$ where

$$\delta^{(l)}_j = \frac{ \partial{C} } { \partial{z^{(l)}_j} }$$

for any arbitrary layer $l$. This new $\delta$ covers the two last partial derivatives in the chain of partial derivatives shown above.

For the output layer $L$ the following is true.

$$\delta^{(L)}_j = \frac{ \partial{C} } { \partial{z^{(L)}_j} }= \frac{ \partial{C} }{\partial{ a^{(L)}_j} }\frac{ \partial{a^{(L)}_j} } { \partial{z^{(L)}_j} }$$
$$= (a^{(L)}_j-y_{j})*\sigma\prime(z^{(L)}_j)$$ 
The above component-wise equation can be condensed using matrix multiplicaton into the following form,

$$\delta^{(L)} = (a^{(L)}-y)\odot \sigma\prime(z^{(L)})$$

where $\odot$ is the hadamard operator.

For any arbitrary layer and node $l$ and $j$, $\delta^{(l)}_j$ can be computed recursively using the values of $\delta^{(l+1)}_k$ for every $k$th node in the $l+1$th layer. To understand the following derivation, it's important to note a very relevant nontrivial fact: any node in any arbitrary layer that is not the output layer affects the cost function through multiple paths.
<center>
    <img src="diagrams/IMG_4956.png" width="300" alt="multiple paths"/>
</center>

(IMG_4956)

In the simplified diagram above, node $a^{(l)}_1$ indirectly affects the cost function by affecting the two nodes it directly affects -- $a^{(l+1)}_1$ and $a^{(l+1)}_2$. Thus, $\frac{\partial {C} }{\partial {a^{(l)}_1} }$ can be said to be equal to 
$$\frac{\partial {C} }{\partial {a^{(l+1)}_1} }\frac{\partial {a^{(l+1)}_1} }{\partial {z^{(l+1)}_1} } \frac{\partial {z^{(l+1)}_1} }{\partial {a^{(l)}_1} }+
\frac{\partial {C} }{\partial {a^{(l+1)}_2} }\frac{\partial {a^{(l+1)}_2} }{\partial {z^{(l+1)}_2} } \frac{\partial {z^{(l+1)}_2} }{\partial {a^{(l)}_1} }$$

In general,

$$\frac{\partial {C} }{\partial {a^{(l)}_j} } $$
$$= \sum_{k=1}^{N}[ \frac{\partial {C} }{\partial {a^{(l+1)}_k} }\frac{\partial {a^{(l+1)}_k} }{\partial {z^{(l+1)}_k} } \frac{\partial {z^{(l+1)}_k} }{\partial {a^{(l)}_j} }]$$
$$= \sum_{k=1}^{N} [\frac{\partial {C} }{\partial {z^{(l+1)}_k} } \frac{\partial {z^{(l+1)}_k} }{\partial {a^{(l)}_j} }]
$$

With this in mind, computing $\delta^{(l)}_j$ can be done using the chain rule.

$$\delta^{(l)}_j = \frac{ \partial{C} } { \partial{z^{(l)}_j} }= \frac{ \partial{C} }{ \partial{a^{(l)}_j} }\frac{ \partial{a^{(l)}_j} } { \partial{z^{(l)}_j} } $$

$$= 
\frac{ \partial{a^{(l)}_j} } { \partial{z^{(l)}_j} }
\sum_{k=1}^{N}[
\frac{ \partial{C} } { \partial{z^{(l+1)}_k} }
\frac{ \partial{z^{(l+1)}_k} }{ \partial{a^{(l)}_j} }]$$

$$
=\frac{ \partial{a^{(l)}_j} } { \partial{z^{(l)}_j} }
\sum_{k=1}^{N}[
\delta^{(l+1)}_k*
\frac{\partial}{\partial{a^{(l)}_j}}[\sum_{z=1}^{N} a^{(l)}_z*w^{(l+1)}_{kz} + b^{(l+1)}]]
$$

$$=\frac{ \partial{a^{(l)}_j} } { \partial{z^{(l)}_j} }
\sum_{k=1}^{N}
\delta^{(l+1)}_k*w^{(l+1)}_{kj}
$$ 

$$=\sigma\prime(z^{(l)}_j)
\sum_{k=1}^{N}
\delta^{(l+1)}_k*w^{(l+1)}_{kj}
$$

In condensed matrix multiplication form, the above component-wise form can be written more simply and elegantly as

$$\delta^{(l)} = ((w^{(l+1)})^T\delta^{(l+1)})\odot \sigma\prime(z^{(l)}).$$

Finally, we can come up with a general expression for any partial derivative of the cost function with respect to any weight or bias in the network. 

$$\frac{\partial {C} }{\partial {w^{(l)}_{jk}} } = \frac{\partial {z^{(l)}_{j}} }{\partial {w^{(l)}_{jk}} } \frac{\partial {C} }{\partial {z^{(l)}_{j}} }$$
$$=a^{(l-1)}_k*\delta^{(l)}_j$$
$$\frac{\partial {C} }{\partial {b^{(l)}_j }} = \frac{\partial {z^{(l)}_{j}} }{\partial {b^{(l)}_j} } \frac{\partial {C} }{\partial {z^{(l)}_{j}} }$$
$$=\delta^{(l)}_j$$

In condensed matrix form, 

$$\nabla_w^{(l)} = \delta^{(l)}A^{(l-1)T}$$
$$\nabla_b^{(l)} = \delta^{(l)}$$

All of this math only applies for one test case. In order to simultaneously go through all of the test cases, the gradients should be divided by the number of test cases.

### Thoughts

It seems best not to get caught up in the confusing matrix multiplication forms of the relatively simple component-wise equations. As I noted earlier, focusing on the dimensions of the matrices seems to be enough for now.

In [7]:
W1,b1,W2,b2 = gradDesc(X_train,Y_train,0.1,100)
testWB(W1,b1,W2,b2,X_dev,Y_dev)

accuracy for test data: 0.627
